# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

  - 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies

We'll first install all our required libraries.

In [1]:
!pip install -qU langchain langchain_openai langchain-community langgraph arxiv duckduckgo_search==5.3.1b1

## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE4 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Duck Duck Go Web Search](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/ddg_search)
- [Arxiv](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/arxiv)

####🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [4]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tool_belt = [
    DuckDuckGoSearchRun(),
    ArxivQueryRun(),
]

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [18]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [19]:
model = model.bind_tools(tool_belt)

#### ❓ Question #1:
How does the model determine which tool to use?


When the model gets a user input, it figures out what kind of task is needed by analyzing the text. Based on the context and the tools it has in its tool belt, like the DuckDuckGo search or Arxiv paper search, it decides which one to use. This decision is influenced by how the tools are set up and the model’s training, which helps it recognize when a specific tool is the right fit for the job.

Once the model picks the tool, it calls it up to do the work, like finding information or retrieving data. The tool does its thing (as Chris would say!), and the model then uses the results to craft the final response.

The model isn’t directly running Python code or invoking tools itself. Instead, it generates outputs that suggest what tools should be used, and the surrounding system handles the execution of those tools.

## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [20]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]
  cycle_count: int  # Adding a counter to track cycles

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [21]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  # Ensure the cycle_count is initialized
  if "cycle_count" not in state:
    state["cycle_count"] = 0
  
  messages = state["messages"]
  response = model.invoke(messages)

  # Increment the cycle count after processing
  state["cycle_count"] += 1

  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [22]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [23]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [24]:
def should_continue(state):
  last_message = state["messages"][-1]

  # Check if the cycle limit has been reached
  if state.get("cycle_count", 0) >= 10:  # Cycle limit
    return END

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [25]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [26]:
compiled_graph = uncompiled_graph.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

If not, how could we impose a limit to the number of cycles?

There isn’t a built-in limit to how many times we can cycle through nodes in a workflow. While this provides a lot of flexibility, it also means that without any safeguards, a workflow could potentially loop forever if the conditions for continuing the loop are always met.

This can be avoided by adding a cycle limit through a counter in the state object. This notebook has been modified to include this feature.

## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [27]:
from langchain_core.messages import HumanMessage

# Initialize the inputs with the cycle_count set to 0
inputs = {
    "messages": [HumanMessage(content="Who is the current captain of the Winnipeg Jets?")],
    "cycle_count": 0  # Initializing cycle_count here
}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WahdEh5bloLbjxUG1Znlod9L', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets 2023"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 156, 'total_tokens': 181}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d6deef53-4a77-4ac7-bfb6-52f311c25e51-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current captain of the Winnipeg Jets 2023'}, 'id': 'call_WahdEh5bloLbjxUG1Znlod9L', 'type': 'tool_call'}], usage_metadata={'input_tokens': 156, 'output_tokens': 25, 'total_tokens': 181})]



Receiving update from node: 'action'
[ToolMessage(content='The Winnipeg Jets will have a captain for the 2023-24 season. After going captain-less in 2022-23, the Winnipeg Jets unveiled 

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [28]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the QLoRA paper, then search each of the authors to find out their latest Tweet using DuckDuckGo.")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PEoi6GkzYbpAzCNChpPxjrnD', 'function': {'arguments': '{"query": "QLoRA"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_FIaUESayTuPtOvZTiyYqOKyi', 'function': {'arguments': '{"query": "latest Tweet Tim Dettmers"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_4Pdo2jbdPvdZd8G2nnRxYdVC', 'function': {'arguments': '{"query": "latest Tweet Mike Lewis"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_FIkQCcNnv1VMA4EzF3JsLrS6', 'function': {'arguments': '{"query": "latest Tweet Sam Shleifer"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_QI8mgBvXvep0ZqgQoBUOkiWC', 'function': {'arguments': '{"query": "latest Tweet Luke Zettlemoyer"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 173, 'total_tokens': 292}, 'model_name': 'gpt

####🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

1. Initial Request: The state object was initialized with the user’s query asking to "Search Arxiv for the QLoRA paper, then search each of the authors to find out their latest Tweet using DuckDuckGo."
2. State Passed to Agent Node: The state object, containing the initial request, was passed to the entry point (agent node). The agent node processed the request and recognized that the query involves multiple actions (searching Arxiv and DuckDuckGo).
3. First Tool Invocation (Arxiv): The agent node added a message indicating a tool call for Arxiv to the state object and passed it along the conditional edge to the action node. The action node then executed the Arxiv search tool and added the results to the state.
4. Loop Back to Agent Node: After performing the Arxiv search, the action node passed the updated state back to the agent node. The agent node reviewed the updated state and recognized that there was still an outstanding task (searching DuckDuckGo).
5. Second Tool Invocation (DuckDuckGo): The agent node added another message indicating a tool call for DuckDuckGo to the state object and passed it along the conditional edge to the action node again. The action node executed the DuckDuckGo search tool, retrieved the latest tweets of the authors, and added these results to the state.
6. Final State Update: After the DuckDuckGo tool completed its task, the action node passed the final updated state back to the agent node. The agent node added the final response to the state object and, since there were no more tasks to complete, passed the state to the END node.
7. Completion: The process completed successfully, with the final output including both the results from the Arxiv search and the DuckDuckGo search, demonstrating the agent’s ability to handle multiple tools in sequence based on a single complex user query.

## Part 2: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [29]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain = convert_inputs | compiled_graph | parse_output

In [30]:
agent_chain.invoke({"question" : "What is RAG?"})

'RAG stands for Retrieval-Augmented Generation. It is a technique used in natural language processing (NLP) and machine learning to improve the performance of language models by combining retrieval-based methods with generative models. Here’s a brief overview of how it works:\n\n1. **Retrieval**: In the first step, the system retrieves relevant documents or pieces of information from a large corpus or database. This is typically done using a retrieval model, such as BM25 or a dense retrieval model like DPR (Dense Passage Retrieval).\n\n2. **Augmentation**: The retrieved documents are then used to augment the input to the generative model. This means that the generative model has access to additional context or information that can help it produce more accurate and relevant responses.\n\n3. **Generation**: Finally, the generative model (such as GPT-3 or BERT) uses the augmented input to generate a response. The additional context provided by the retrieved documents helps the model gener

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]
```

####🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions.

In [31]:
questions = [
    "What are the differences between CSS Grid and Flexbox?",
    "How do you create a responsive design using media queries?",
    "What is the purpose of the 'position' property in CSS?",
    "How can you center a div both horizontally and vertically using CSS?",
    "What are CSS Grid template areas, and how are they used?",
    "How does the 'auto-fill' value work in CSS Grid layout?"
]

answers = [
    {"must_mention" : ["flexible", "one-dimensional", "two-dimensional"]},
    {"must_mention" : ["media queries", "breakpoints", "responsive"]},
    {"must_mention" : ["relative", "absolute", "fixed", "sticky"]},
    {"must_mention" : ["flexbox", "grid", "align-items", "justify-content"]},
    {"must_mention" : ["grid-template-areas", "named areas", "layout"]},
    {"must_mention" : ["auto-fill", "grid-template-columns", "responsive grid"]}
]

Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [32]:
from langsmith import Client

client = Client()
dataset_name = f"Retrieval Augmented Generation - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about advanced layouts with CSS to Evaluate RAG over this topic."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    outputs=answers,
    dataset_id=dataset.id,
)

#### ❓ Question #3:

How are the correct answers associated with the questions?

> NOTE: Feel free to indicate if this is problematic or not

The correct answers are matched with the questions based on their order in the lists. So, the first question is paired with the first answer, the second question with the second answer, and so on. This method is easy to set up and works fine when the dataset is small and straightforward. But it does have a downside—if the order gets mixed up, the wrong answers might get paired with the wrong questions, leading to incorrect evaluations. Plus, as the list grows, keeping everything in the right order can become a bit of a hassle.

A more reliable approach could be to use explicit pairing, where each question is directly linked to its correct answer in a dictionary. This way, even if the order changes, each question will still have the right answer attached to it. It’s a bit more work upfront, but it makes the evaluation process more robust and less prone to errors, especially when dealing with larger or more complex datasets.

### Task 2: Adding Evaluators

Now we can add a custom evaluator to see if our responses contain the expected information.

We'll be using a fairly naive exact-match process to determine if our response contains specific strings.

In [33]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

The must_mention evaluator you've set up is straightforward and effective for basic checks, but there are several ways to enhance its functionality. One current limitation is the strict matching approach, where every required phrase must be present in the prediction for it to pass. This could be improved by implementing partial matching or fuzzy matching, using a library like fuzzywuzzy. This would allow the evaluator to recognize synonymous or similarly structured phrases as valid matches, making the evaluation more flexible.

Another area for improvement is assessing the contextual relevance of the key phrases. Currently, the evaluator only checks for the presence of phrases, but it doesn't determine whether they are used correctly or meaningfully. Enhancing the evaluator to analyze the surrounding words or using a more advanced language model could help ensure that key phrases are not just mentioned, but also applied appropriately in context.

The current setup treats all required phrases equally, which might not reflect their true importance in the context of the answer. Introducing a weighting system, where some phrases carry more weight than others, could provide a more nuanced evaluation. For example, essential phrases could have a higher impact on the final score, rather than a simple binary pass/fail system.

Also, the evaluator could be improved by handling negations and ambiguities better. Right now, it doesn't account for cases where a phrase is mentioned in a negating or contradictory manner, such as "X is not important" versus "X is important." Implementing logic to detect these situations, possibly through sentence structure analysis or sentiment analysis, would make the evaluation more accurate. Additionally, introducing criteria to assess the overall quality of the response, such as its length, conciseness, and structure, would ensure that the agent provides not just correct, but well-formed answers.

Now that we have created our custom evaluator - let's initialize our `RunEvalConfig` with it!

In [36]:
from langchain.smith import RunEvalConfig, run_on_dataset

eval_config = RunEvalConfig(
    custom_evaluators=[must_mention],
)

Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [35]:
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=agent_chain,
    evaluation=eval_config,
    verbose=True,
    project_name=f"RAG Pipeline - Evaluation - {uuid4().hex[0:8]}",
    project_metadata={"version": "1.0.0"},
)

View the evaluation results for project 'RAG Pipeline - Evaluation - 706efb33' at:
https://smith.langchain.com/o/55f39e46-bf14-5a73-afb7-c39afc25b44b/datasets/83682756-3794-4e35-9043-99e56ff53b36/compare?selectedSessions=55f8f793-69f7-4ddd-bf1a-510d95f1d2cc

View all tests for Dataset Retrieval Augmented Generation - Evaluation Dataset - e2e8ca8d at:
https://smith.langchain.com/o/55f39e46-bf14-5a73-afb7-c39afc25b44b/datasets/83682756-3794-4e35-9043-99e56ff53b36
[------------------------------------------------->] 6/6

,feedback.must_mention,error,execution_time,run_id
count,6,0,6.000000,6
unique,2,0,NaN,6
top,True,NaN,NaN,83b39819-7d83-413a-b786-8c78913cf5ea
freq,5,NaN,NaN,1
mean,NaN,NaN,8.964740,NaN
std,NaN,NaN,2.099104,NaN
min,NaN,NaN,5.948715,NaN
25%,NaN,NaN,7.477056,NaN
50%,NaN,NaN,9.822384,NaN
75%,NaN,NaN,10.397398,NaN


{'project_name': 'RAG Pipeline - Evaluation - 706efb33',
 'results': {'51bf3b9a-3980-4f29-bf04-30eaa088e597': {'input': {'question': 'How can you center a div both horizontally and vertically using CSS?'},
   'feedback': [EvaluationResult(key='must_mention', score=False, value=None, comment=None, correction=None, evaluator_info={}, feedback_config=None, source_run_id=UUID('1850a3e3-fa84-4a88-ba94-42707fc395bc'), target_run_id=None)],
   'execution_time': 5.948715,
   'run_id': '83b39819-7d83-413a-b786-8c78913cf5ea',
   'output': 'To center a `div` both horizontally and vertically using CSS, you can use several methods. Here are a few common approaches:\n\n### 1. Using Flexbox\nFlexbox is a modern and flexible way to center elements.\n\n```css\n.container {\n  display: flex;\n  justify-content: center; /* Center horizontally */\n  align-items: center;    /* Center vertically */\n  height: 100vh;          /* Full viewport height */\n}\n\n.centered-div {\n  /* Your div styles */\n}\n```\n

## Part 3: LangGraph with Helpfulness:

### Task 1: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [37]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

####🏗️ Activity #5:

Please write markdown for the following cells to explain what each is doing.

##### YOUR MARKDOWN HERE
Here we are creating a new Graph called "graph_with_helpfulness_check" and are adding two Nodes to it (agent and action).

In [38]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

##### YOUR MARKDOWN HERE
Here we are telling the Graph which Node will be the entry point (the first node to be called when there is an user input).

In [39]:
graph_with_helpfulness_check.set_entry_point("agent")

##### YOUR MARKDOWN HERE
Here we use conditional logic to determine whether the agent should continue looping or stop. This includes a helpfulness check and a cycle limit.

In [41]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return "END"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4")

  helpfulness_chain = prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

####🏗️ Activity #4:

Please write what is happening in our `tool_call_or_helpful` function!

It checks whether the last message from the agent triggered a tool call or whether the response was helpful. If the response is deemed helpful, the process ends. If not, it continues, but only until the loop limit (e.g., 10 cycles) is reached.

##### YOUR MARKDOWN HERE
This code is adding conditional edges that basically route the flow back to the Agent if the function returns "continue", or moves to the Action node if it returns "action" so that a tool might be used. Otherwise, it will terminate the cycle.

In [42]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

##### YOUR MARKDOWN HERE
Here an edge or connector is added between the Action and the Agent nodes.

In [43]:
graph_with_helpfulness_check.add_edge("action", "agent")

##### YOUR MARKDOWN HERE
Now that the Graph is ready, we proceed to compile it so that it can be used.

In [44]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

##### YOUR MARKDOWN HERE
Now we test our new cycles limit and helpfullness logic by passing the Graph a user input to trigger the process.

In [45]:
inputs = {"messages" : [HumanMessage(content="Related to machine learning, what is LoRA? Also, who is Tim Dettmers? Also, what is Attention?")]}

async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_omaXZSHD3h8xWicGb5B5J5XG', 'function': {'arguments': '{"query": "LoRA machine learning"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_3N8Wq91qQK0PxVZfSAC6e3jq', 'function': {'arguments': '{"query": "Tim Dettmers"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_dQcicleGeV2AyP6WYkiatKdI', 'function': {'arguments': '{"query": "Attention in machine learning"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 171, 'total_tokens': 247}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3de33e75-3fe1-40ff-969a-e7c50d96d8a9-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'LoRA machine learning'}, 'id': 'call_omaXZSHD3h8xWicGb5B5J5XG', 'type': 'tool_call'}, 

### Task 2: LangGraph for the "Patterns" of GenAI

Let's ask our system about the 4 patterns of Generative AI:

1. Prompt Engineering
2. RAG
3. Fine-tuning
4. Agents

In [46]:
patterns = ["prompt engineering", "RAG", "fine-tuning", "LLM-based agents"]

In [47]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

Prompt engineering is a concept primarily associated with the field of artificial intelligence, particularly in the context of natural language processing (NLP) and large language models like GPT-3. It involves the design and crafting of prompts (input text) to elicit desired responses from AI models. The goal is to optimize the input to get the most accurate, relevant, or useful output from the model.

### Key Aspects of Prompt Engineering:
1. **Crafting Effective Prompts**: Designing prompts that guide the AI to produce the desired output.
2. **Iterative Testing**: Continuously refining prompts based on the responses received.
3. **Understanding Model Behavior**: Knowing how different models respond to various types of input.
4. **Use Cases**: Applications range from chatbots and virtual assistants to content generation and data analysis.

### Emergence of Prompt Engineering:
Prompt engineering became more prominent with the advent of large-scale language models like OpenAI's GPT-3, 